# Traffic classification - DDoS or benign.

Log research of IDS systems (Intrusion Detection System) for 
detection of abnormal traffic using machine learning.

DDoS data extracted from different public IDS's and concatenated with benign (normal) traffic flows. 

Base datasets:
1. CSE-CIC-IDS2018-AWS: https://www.unb.ca/cic/datasets/ids-2017.html
2. CICIDS2017: https://www.unb.ca/cic/datasets/ids-2018.html
3. CIC DoS dataset(2016): https://www.unb.ca/cic/datasets/dos-dataset.html

## Download dataset. Import libraries.

In [12]:
! pip install kaggle -q

In [13]:
! kaggle datasets download -d devendra416/ddos-datasets -f ddos_balanced/final_dataset.csv -p ./content/ --unzip

Dataset URL: https://www.kaggle.com/datasets/devendra416/ddos-datasets
License(s): other
final_dataset.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [14]:
import zipfile
import os

if not os.path.exists('./content/final_dataset.csv'):
    print('Extracting dataset...')
    with zipfile.ZipFile('./content/final_dataset.csv.zip', 'r') as zip_ref:
        zip_ref.extractall('./content/')
    print('Success.')
else:
    print('Dataset already extracted.')

Dataset already extracted.


In [15]:
# Libraries.
